# This notebook includes a simple demo for flood risk assessment

## It uses flood hazard water depth scenario RP100 and population map for 2018

In [1]:
# Added to requirements.txt, but need to specify version? Then remove here
!pip -q install geopandas
!apt -q install libspatialindex-dev
!pip -q install osmnx
!pip -q install rasterio
!pip -q install rasterstats
!pip -q install geojson

Reading package lists...
Building dependency tree...
Reading state information...
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [6]:
!git clone https://github.com/gfdrr/rdl-pytools.git

Cloning into 'rdl-pytools'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 62 (delta 6), reused 13 (delta 5), pack-reused 46
Unpacking objects: 100% (62/62), done.


### Import required libraries

In [36]:
# Required libraries
import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio
import rasterio.mask
from rasterio.enums import Resampling
from rasterio.plot import show

from rasterstats import zonal_stats, utils

import gdal, gdalconst
from osgeo import ogr, osr
from shapely import wkb
from matplotlib import pyplot
from shapely.geometry import shape



Do we need this class stuff?

In [8]:
# #class to work with tiff datasets
# class Raster:
    
#     def __init__(self, filename):
#         self.filename = filename
#     #get data using bounding box
#     def dataFromBounds(self, band, bounds):
#         with rasterio.open(self.filename) as src:
#             self.noData = src.nodatavals[0]
#             window = rasterio.windows.from_bounds(bounds[0],bounds[1],bounds[2],bounds[3], src.transform)
#             self.boundsData = src.read(band, window=window)
#             self.boundsTransform = src.window_transform(window)
#     #get data using polygon for masking
#     def dataFromShape(self,band,shape):
#         with rasterio.open(self.filename) as src:
#             self.noData = src.nodatavals[0]
#             self.maskData, self.maskTransform = rasterio.mask.mask(src, [shape], crop=True, nodata=0)
#             self.maskData = self.maskData[band-1, :, :]


### Load input data: flood hazard (RP100) and population (Landscan 2018)

All data already share the same crs (epsg 4326)

In [54]:
base_loc = "rdl-pytools/data/ssudan/"

#pop represents population data
with rasterio.open(base_loc+'exp-ss-pop-landscan18.tif') as fp:
  pop_data = fp.read(1)


scale_factor = 0.1
with rasterio.open(base_loc+'hzd-ss-fl-rp100.tif') as dataset:
  # resample data to target shape
  data = dataset.read(
      out_shape=(
          dataset.count,
          int(dataset.height * scale_factor),
          int(dataset.width * scale_factor)
      ),
      resampling=Resampling.bilinear
  )

  # scale image transform
  transform = dataset.transform * dataset.transform.scale(
      (dataset.width / data.shape[-1]),
      (dataset.height / data.shape[-2])
  )

flood_data = data



In [55]:
flood_extent = gpd.read_file(base_url+"hzd-ss-fl-rp100.shp")

DriverError: ignored

In [50]:
dataset.width, dataset.height

(21000, 24000)

In [52]:
fp.width, fp.height

(1416, 1050)

In [46]:
flood_data.shape

(1, 2400, 2100)

In [48]:
pop_data.shape

(1050, 1416)

In [47]:
#adm2 represents administrative units
adm0 = gpd.read_file(base_loc+'SS_ADM0.shp')
adm1 = gpd.read_file(base_loc+'SS_ADM1.shp')
adm2 = gpd.read_file(base_loc+'SS_ADM2.shp')

## 1. Flooded area

In [26]:
target_adm = adm1

flood_crs = "EPSG:4326"

target = target.to_crs(flood_crs)

# flood_dict = {
#     0:'non-flooded',
#     1:"flooded"
# }

target

# zs = pd.DataFrame(zonal_stats(target, flood_data, affine=meta['transform'], categorical=True, category_map=flood_dict))
# area.head()

,ADM0_EN,ADM0_PCODE,ADM0_REF,po18sum,sum,geometry
0,South Sudan,SS,None,1.436904e+07,6.037584e+06,"POLYGON ((33.89957 10.16584, 33.90147 10.16155..."


### Export output

In [27]:
# area.to_csv(os.path.join(out_folder, "area_flooded.csv"))

In [28]:
# out_folder = os.path.join(data_dir, "output")
# if not os.path.exists(out_folder):
#     os.mkdir(out_folder)

## 2. Flooded population

### Create flood hazard extent mask using a threshold of 15 cm

In [37]:
# Get flooded areas
flood_mask = np.where(flood_data > 0.15, 1, 0)

In [41]:
flood_mask.shape

(24000, 21000)

In [42]:
pop.shape

(1050, 1416)

### Estimate population within hazard extent and plot as map

In [40]:
pop_fd = flood_mask * pop

ValueError: ignored

In [ ]:
#print total population affected
totalPop = 0
for el in zonalStatsFB:
  if not el['sum'] is None:
    totalPop = totalPop + el['sum']

print(totalPop)

In [ ]:
plot pop_fd over osm basemap

### Summary for ADM2 level - zonal statistics

In [ ]:
# total population affected
totalPop = 0
for el in zonalStatsFB:
  if not el['sum'] is None:
    totalPop = totalPop + el['sum']

print(totalPop)

In [ ]:
# population affected for each ADM2 unit
target = target.to_crs(pop_crs)
zs_landscan = pd.DataFrame(zonal_stats(target, pop_fd, affine=landscan.meta['transform'], stats='sum')).rename(columns = {'sum':'pop_flooded_rp100'})

plot = results shown as map and as table

### Export output

In [ ]:
area.to_csv(os.path.join(out_folder, "pop_flooded_rp100.csv"))

In [ ]:
out_folder = os.path.join(data_dir, "output")
if not os.path.exists(out_folder):
    os.mkdir(out_folder)